In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.callbacks import LearningRateScheduler

In [ ]:
# Define paths
dataset_path = os.path.abspath('../dataset/emotion-recog-dataset')
emotion_labels = ['Angry', 'Happy', 'Neutral', 'Sad', 'Surprise']

In [ ]:
# Load the dataset
def load_emotion_data(dataset_path, emotion_labels, img_size=(128, 128)):
    images = []
    labels = []
    
    for label in emotion_labels:
        emotion_path = os.path.join(dataset_path, label)
        print(f"Looking for images in: {emotion_path}")  # Debug print
        if not os.path.exists(emotion_path):
            print(f"Directory does not exist: {emotion_path}")
            continue
        for filename in os.listdir(emotion_path):
            img_path = os.path.join(emotion_path, filename)
            print(f"Processing image: {img_path}")  # Debug print
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)  # Read as grayscale
            if img is None:
                print(f"Failed to read image: {img_path}")
                continue
            img = cv2.resize(img, img_size)  # Resize to (128, 128)
            images.append(img)
            labels.append(emotion_labels.index(label))  # Assign a numerical label based on the emotion
            
    images = np.array(images)
    labels = np.array(labels)
    
    # Normalize the images
    images = images / 255.0
    
    # Reshape images to add channel dimension
    images = images.reshape(-1, img_size[0], img_size[1], 1)
    
    # Convert labels to one-hot encoding
    labels = to_categorical(labels, num_classes=len(emotion_labels))
    
    return images, labels

In [ ]:
# Load data
X, y = load_emotion_data(dataset_path, emotion_labels);

In [ ]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Define the input shape
input_shape = (128, 128, 1)

In [ ]:
# Model architecture
inputs = Input(shape=input_shape)

x = Conv2D(32, (3, 3), activation='relu')(inputs)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Dropout(0.25)(x)  # Add dropout after first pooling layer

x = Conv2D(64, (3, 3), activation='relu')(x)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Dropout(0.25)(x)  # Add dropout after second pooling layer

x = Conv2D(128, (3, 3), activation='relu')(x)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Dropout(0.25)(x)  # Add dropout after third pooling layer

x = Conv2D(256, (3, 3), activation='relu')(x)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Dropout(0.25)(x)  # Add dropout after fourth pooling layer

x = Conv2D(512, (3, 3), activation='relu')(x)
x = BatchNormalization()(x)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Dropout(0.3)(x)

x = Flatten()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)  # Increase dropout rate
outputs = Dense(len(emotion_labels), activation='softmax')(x)

model = Model(inputs=inputs, outputs=outputs)

# Compile the model
model.compile(
    loss='categorical_crossentropy',
    optimizer=Adam(learning_rate=0.0001),
    metrics=['accuracy']
)

# Learning Rate Scheduler
def scheduler(epoch, lr):
    if epoch > 50:
        return lr * 0.5
    elif epoch > 30:
        return lr * 0.75
    return lr

lr_scheduler = LearningRateScheduler(scheduler)

# Updated Callbacks
callbacks = [
    EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-7),
    ModelCheckpoint('../models/best_emotion_model.keras', monitor='val_loss', save_best_only=True),
    lr_scheduler
]

In [ ]:
# Train the model
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    batch_size=32,
    epochs=100,
    callbacks=callbacks
)

In [ ]:
# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {test_accuracy * 100:.2f}%')

In [ ]:
# Extract loss values
training_loss = history.history['loss']
validation_loss = history.history['val_loss']

# Create epochs range
epochs = range(1, len(training_loss) + 1)

# Plotting the training and validation loss
plt.figure(figsize=(10, 6))
plt.plot(epochs, training_loss, 'r', label='Training Loss')
plt.plot(epochs, validation_loss, 'b', label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.show()